the overall priority system should be something like:
get nortons.
if fewer than 5 nortons, get sbns
if fewer than 5 sbsn, get strict parens
if fewer than 5 strict parens, get loose parens

#that requires an update to the classes to store strict and loose parens collection groups.


return all citations. 